## CMBAgent - OSS Testing

*For testing purposes only, to be removed before opening a PR to* `main` .

In [ ]:
# Local model testing supported via https://github.com/acceleratescience/mini-vllm (private Accelerate repo)

import os
from dotenv import load_dotenv

load_dotenv()

BASE_URL = os.getenv("LOCAL_LLM_BASE_URL")
API_KEY = os.getenv("LOCAL_LLM_API_KEY")

local_llm_config = {
    "config_list": [
        {
            "model": "openai/gpt-oss-20b", # Must match model id from endpoint
            "api_key": API_KEY,
            "api_type": "openai",
            "base_url": BASE_URL,
            "price": [0.0, 0.0], # Passed explicitly to ignore warning
        },
    ],
    "cache_seed": None
}

### AG2 ConversableAgent() with `openai/gpt-oss-20b`

Simple chat with a tiny model (`openai/gpt-oss-20b`)to verify ConversableAgent() has OSS

In [ ]:
from autogen import UserProxyAgent, ConversableAgent

# Creates the agent that uses the vLLM LM
assistant = ConversableAgent(
    "agent",
    llm_config=local_llm_config,
    system_message="Answer briefly.",
)

# Create the agent that represents the user in the conversation
user_proxy = UserProxyAgent(
    "user",
    code_execution_config=False,
    system_message="",
    human_input_mode="NEVER",
)

# User asks a question
assistant.initiate_chat(
    user_proxy,
    message="What is the meaning of Denario?",
    max_turns=2,
)

agent (to user):

What is the meaning of Denario?

--------------------------------------------------------------------------------
user (to agent):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000             86                312           398
agent (to user):

*Denario* is the Latin word for a Roman silver coin that entered use in 211 BC, worth 10 asses. Its name comes from the Latin “denarius,” itself derived from a unit of weight, and in modern Romance languages the word (and its cognates) still means “money” or “coin.”

--------------------------------------------------------------------------------
user (to agent):



--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (fbd17823-fc2b-4f15-a978-ab0b3c69f06a): Maximum turns (2) reached


ChatResult(chat_id=142075004582337632179642004039302304976, chat_history=[{'content': 'What is the meaning of Denario?', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}, {'content': '*Denario* is the Latin word for a Roman silver coin that entered use in 211\u202fBC, worth 10 asses. Its name comes from the Latin “denarius,” itself derived from a unit of weight, and in modern Romance languages the word (and its cognates) still means “money” or “coin.”', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0.0, 'openai/gpt-oss-20b': {'cost': 0.0, 'prompt_tokens': 86, 'completion_tokens': 312, 'total_tokens': 398}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'openai/gpt-oss-20b': {'cost': 0.0, 'prompt_tokens': 86, 'completion_tokens': 312, 'total_tokens': 398}}}, human_input=[])

### AG2 ConversableAgent() Tool Calling with OSS

In [ ]:
def add(a: float, b: float) -> float:
    return a + b

assistant.register_for_llm(name="add", description="Add two numbers")(add)
assistant.register_for_execution(name="add")(add)

result = assistant.initiate_chat(
    recipient=assistant,
    message="Use the add tool to add 2 and 3. Then reply with just the number.",
    max_turns=3,
)

for msg in result.chat_history:
    if msg.get("tool_calls"):
        print(msg["tool_calls"])

agent (to agent):

Use the add tool to add 2 and 3. Then reply with just the number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000            175                 39           214
agent (to agent):

***** Suggested tool call (chatcmpl-tool-b17d29ad9f94de4a): add *****
Arguments: 
{"a": 2, "b": 3}
*********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION add...
Call ID: chatcmpl-tool-b17d29ad9f94de4a
Input arguments: {'a': 2, 'b': 3}

>>>>>>>> EXECUTED FUNCTION add...
Call ID: chatcmpl-tool-b17d29ad9f94de4a
Input arguments: {'a': 2, 'b': 3}
Output:
5.0
agent (to agent):

***** Response from calling Tool tool (chatcmpl-tool-b17d29ad9f94de4a) *****
5.0
**********

### OSS with Engineer Only

In [ ]:
from pathlib import Path
from cmbagent.workflows import one_shot

local_engineer_config = local_llm_config["config_list"][0]

from dotenv import load_dotenv

load_dotenv()

tmp_path = Path.cwd()

task = """
Plot the graph of y = x from x = 0 to 1
"""

results = one_shot(
    task,

    max_rounds=5,
    agent="engineer",
    engineer_model=local_engineer_config,
    work_dir=str(tmp_path / "one_shot"),
)

[OSS] Defaulted formatter model to openai/gpt-oss-20b
_User (to main_cmbagent_chat):


Plot the graph of y = x from x = 0 to 1


--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
             Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b engineer 0.00000           1768               1540          3308
engineer (to main_cmbagent_chat):

**Code Explanation:**

The script creates a plot of the function y = x over the interval [0, 1]. It generates 100 evenly spaced points, plots the line, labels the axes, adds a title and grid, and saves the figure as a high‑resolution PNG file in the `data/` directory. The filename includes a timestamp to ensure uniqueness. The script prints a concise message indicating where the plot was saved.

**Modifications:** None.

**Python Code:**

```python
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt

p